# Data Exploration

This file was used to gain an overview on the provided data, and its parts are later reused to ingest the database upon startup of the server.

## Part 1: Weather Data

In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [3]:
# Read in weather files
weather_files = "../data/wx_data/"
columns = ['date', 'temperature_max*10',
            'temperature_min*10', 'precipitation*10']
weather_files_data = []
for file in glob.glob(os.path.join(weather_files, "*.txt")):
    weather_frame = pd.read_csv(file, delimiter='\t', names=columns)
    weather_frame['stationId'] = file[-12:-4]
    weather_files_data.append(weather_frame)
    weather_data_df = pd.concat(weather_files_data, ignore_index=True)

In [21]:
# Replace -9999 values with np.nan
weather_data_df = weather_data_df.replace(-9999, np.nan)
# Check for duplicates in data
weather_data_df = weather_data_df.drop_duplicates( keep='first', inplace=False, ignore_index=False)

In [22]:
# Divide tenths of millimeter by 10 to arrive at centimeter
weather_data_df['temperature_max_celcius'] = weather_data_df['temperature_max*10'].div(
    10).round(2)
weather_data_df['temperature_min_celcius'] = weather_data_df['temperature_min*10'].div(
    10).round(2)
weather_data_df['precipitation_ml'] = weather_data_df['precipitation*10'].div(
    10).round(2)

In [23]:
weather_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1729957 entries, 0 to 1729956
Data columns (total 10 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   date                     int64         
 1   temperature_max*10       float64       
 2   temperature_min*10       float64       
 3   precipitation*10         float64       
 4   stationId                object        
 5   temperature_max_celcius  float64       
 6   temperature_min_celcius  float64       
 7   precipitation_ml         float64       
 8   dateTime                 datetime64[ns]
 9   year                     int64         
dtypes: datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 145.2+ MB


In [24]:
weather_data_df.describe()

,date,temperature_max*10,temperature_min*10,precipitation*10,temperature_max_celcius,temperature_min_celcius,precipitation_ml,year
count,1.729957e+06,1.695124e+06,1.693601e+06,1.695188e+06,1.695124e+06,1.693601e+06,1.695188e+06,1.729957e+06
mean,1.999234e+07,1.664358e+02,4.504915e+01,2.532215e+01,1.664358e+01,4.504915e+00,2.532215e+00,1.999167e+03
std,8.547952e+04,1.186926e+02,1.076869e+02,7.622815e+01,1.186926e+01,1.076869e+01,7.622815e+00,8.547952e+00
min,1.985010e+07,-3.440000e+02,-4.500000e+02,0.000000e+00,-3.440000e+01,-4.500000e+01,0.000000e+00,1.985000e+03
25%,1.992042e+07,7.200000e+01,-3.300000e+01,0.000000e+00,7.200000e+00,-3.300000e+00,0.000000e+00,1.992000e+03
50%,1.999070e+07,1.830000e+02,5.000000e+01,0.000000e+00,1.830000e+01,5.000000e+00,0.000000e+00,1.999000e+03
75%,2.006122e+07,2.670000e+02,1.390000e+02,5.000000e+00,2.670000e+01,1.390000e+01,5.000000e-01,2.006000e+03
max,2.014123e+07,4.670000e+02,5.060000e+02,4.295000e+03,4.670000e+01,5.060000e+01,4.295000e+02,2.014000e+03


In [13]:
weather_data_df.head()

,date,temperature_max*10,temperature_min*10,precipitation*10,stationId,temperature_max_celcius,temperature_min_celcius,precipitation_ml
0,19850101,-83.0,-144.0,0.0,00257715,-8.3,-14.4,0.0
1,19850102,0.0,-133.0,0.0,00257715,0.0,-13.3,0.0
2,19850103,22.0,-111.0,0.0,00257715,2.2,-11.1,0.0
3,19850104,61.0,-50.0,0.0,00257715,6.1,-5.0,0.0
4,19850105,78.0,-67.0,0.0,00257715,7.8,-6.7,0.0


## Part 2: Weather Statistics Calculation

Weather Statistics are based on an entire year, hence records are being allocated to their respective year.

In [14]:
# Extract year from date string for statistics
weather_data_df['dateTime'] = pd.to_datetime(
    weather_data_df['date'].astype(str), format='%Y%m%d')
weather_data_df['year'] = pd.DatetimeIndex(
    weather_data_df['dateTime']).year
weather_data_df_reduced = weather_data_df[[
    'stationId', 'dateTime', 'temperature_max_celcius', 'temperature_min_celcius', 'precipitation_ml']]
weather_data_df_reduced.columns = [
    'stationId', 'date', 'maxTemp', 'minTemp', 'precipitation']

weather_data_df_reduced['year'] = pd.DatetimeIndex(
    weather_data_df_reduced['date']).year
# complete weather_data_df in one dataframe with year, stationID, temperature_max/min and precipitation

/var/folders/6n/xwwtvdh528z13yw6htdm3t_00000gn/T/ipykernel_81400/421169749.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data_df_reduced['year'] = pd.DatetimeIndex(


In [15]:
# Calculate Average of Temperatures
weather_avg_temperatures = weather_data_df_reduced[[
    'stationId', 'year', 'maxTemp', 'minTemp']]
weather_avg_temperatures = weather_avg_temperatures.groupby(
    ['stationId', 'year']).mean().reset_index()

In [ ]:
# Calculate Sum of Precipitation
weather_sum_precipitation = weather_data_df_reduced[[
    'precipitation', 'stationId', 'year']]

weather_sum_precipitation = weather_sum_precipitation.groupby(
    ['stationId', 'year']).sum().reset_index()

In [16]:
# Join both tables on stationId and year, and rename columns
weatherStatistics_data = pd.merge(weather_avg_temperatures, weather_sum_precipitation, on=[
    "stationId", 'year'], how="outer")
weatherStatistics_data.columns = ['stationId', 'year',
                                  'maxTempAvg', 'minTempAvg', 'precipitationSum']
weatherStatistics_data.head()

,stationId,year,maxTempAvg,minTempAvg,precipitationSum
0,00110072,1985,15.334795,4.326446,780.1
1,00110072,1986,12.696337,2.176190,505.3
2,00110072,1987,17.760274,6.329863,793.6
3,00110072,1988,17.347268,4.534973,541.0
4,00110072,1989,15.651507,3.983562,793.7


In [17]:
weatherStatistics_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4820 entries, 0 to 4819
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   stationId         4820 non-null   object 
 1   year              4820 non-null   int64  
 2   maxTempAvg        4797 non-null   float64
 3   minTempAvg        4797 non-null   float64
 4   precipitationSum  4820 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 225.9+ KB


In [18]:
weatherStatistics_data.describe()

,year,maxTempAvg,minTempAvg,precipitationSum
count,4820.00000,4797.000000,4797.000000,4820.000000
mean,1999.20332,16.611356,4.469036,890.576929
std,8.56004,2.106051,2.182649,280.122471
min,1985.00000,-5.000000,-14.231034,0.000000
25%,1992.00000,15.383380,3.180992,705.375000
50%,1999.00000,16.660274,4.560000,895.600000
75%,2007.00000,17.915152,5.881694,1078.000000
max,2014.00000,31.980952,20.076190,1847.600000
